In [1]:
import os
import anndata
import pandas as pd
import anndata as ad
from FlowCytometryTools import FCMeasurement

ModuleNotFoundError: No module named 'FlowCytometryTools'

### Create functions to load and concatenate data

In [ ]:
def load_data_fcs(path):
    data = FCMeasurement(ID="Sample", datafile=path)
    anndata = ad.AnnData(data.data)
    anndata.obs['drug']= 'Stim_1_Bcell'
    return anndata

In [ ]:
def concatenate_data(path_stim, path_unstim):
    unstim_anndata = load_data_fcs(path_unstim)
    stim_anndata = load_data_fcs(path_stim)
    stim_anndata.obs['drug']= 'Stim_1_Bcell'
    unstim_anndata.obs['drug']= 'Unstim_Bcell'
    combined_anndata = stim_anndata.concatenate(unstim_anndata, batch_key="condition", batch_categories=["stim", "control"])
    return combined_anndata

/Users/MacBook/miniconda3/envs/cellot_env/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


###  Processs our data

In [ ]:
path_unstim='/Users/MacBook/stanford/cellot/datasets/atest_data/controls_1_Surge_3_HCAA_BL_Unstim - Bcells.fcs'
path_stim = "/Users/MacBook/stanford/cellot/datasets/atest_data/controls_1_Surge_3_IC1_IC1_Stim_1 - Bcells.fcs"
combined_anndata = concatenate_data(path_stim, path_unstim)
combined_anndata.write("/Users/MacBook/stanford/cellot/datasets/atest_data/combined_Bcell.h5ad")

In [ ]:
path_unstim_to_predict='/Users/MacBook/stanford/cellot/datasets/surge_prehab_controls - Gated Populations/controls_1_Surge_3_HCBB_BL_Unstim - Bcells.fcs'
unstim_anndata_to_predict = load_data_fcs(path_unstim_to_predict)
unstim_anndata_to_predict.write("/Users/MacBook/stanford/cellot/datasets/atest_data/unstim_Bcell_to_predict.h5ad")


/Users/MacBook/miniconda3/envs/cellot_env/lib/python3.8/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
... storing 'drug' as categorical


### Pipeline for evaluation

In [ ]:
eval_4i=pd.read_csv("/Users/MacBook/stanford/cellot/results/4i/drug-cisplatin/model-cellot/evals_iid_data_space/evals.csv")

In [10]:
print('eval metric available:',eval_4i['metric'].unique())

eval metric available: ['l2-means' 'l2-stds' 'r2-means' 'r2-stds' 'r2-pairwise_feat_corrs'
 'l2-pairwise_feat_corrs' 'mmd' 'enrichment-k50' 'enrichment-k100']


In [12]:
eval_4i.ncells.unique()

array([100, 250, 500])